In [1]:
import torch
from torch import nn
import torch.onnx
import tensorflow as tf 
from onnx_tf.backend import prepare
import onnx

/usr/local/lib/python3.6/site-packages/onnx_tf/common/__init__.py:87: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/site-packages/onnx_tf/common/__init__.py:87: UserWarning: FrontendHandler.get_outputs_names is deprecated. It will be removed in future release. Use node.outputs instead.
  warnings.warn(message)


In [2]:
class ResBlock(nn.Module):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        
        self.resblock = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels, affine=True),
            nn.ReLU(),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels, affine=True),
        )
        
    def forward(self, x):
        out = self.resblock(x)
        return out + x

class MicroResNet(nn.Module):
    def __init__(self):
        super(MicroResNet, self).__init__()
        
        self.downsampler = nn.Sequential(
            nn.Conv2d(3, 4, kernel_size=9, padding=4, stride=2),
            nn.InstanceNorm2d(4, affine=True),
            nn.ReLU(),
            nn.Conv2d(4, 8, kernel_size=3, padding=1, stride=2),
            nn.InstanceNorm2d(8, affine=True),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1, stride=2),
            nn.InstanceNorm2d(16, affine=True),
            nn.ReLU()
        )
        
        self.residual = nn.Sequential(
            ResBlock(16),
            ResBlock(16)
        )
        
        self.segmentator = nn.Sequential(
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.InstanceNorm2d(8, affine=True),
            nn.ReLU(),
            nn.Conv2d(8, 1, kernel_size=9, padding=4),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out = self.downsampler(x)
        out = self.residual(out)
        out = self.segmentator(out)
        return out

In [3]:
model = MicroResNet()

checkpoint = torch.load('saliency_model_v6.pt')
model.load_state_dict(checkpoint)
model.eval()

MicroResNet(
  (downsampler): Sequential(
    (0): Conv2d(3, 4, kernel_size=(9, 9), stride=(2, 2), padding=(4, 4))
    (1): InstanceNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (2): ReLU()
    (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (5): ReLU()
    (6): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (8): ReLU()
  )
  (residual): Sequential(
    (0): ResBlock(
      (resblock): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): ReLU()
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): InstanceNorm2d(16, eps=1e-05, momentum=0.1, af

In [4]:
class MicroResNetIOS(nn.Module):
    def __init__(self):
        super(MicroResNetIOS, self).__init__()
        self.micro_res_net = model
        
    def forward(self, x):
        x = x.permute(0,3,1,2)
        out = self.micro_res_net(x)
        return out.permute(0,2,3,1)

model_ios = MicroResNetIOS()

In [7]:
with torch.no_grad():
    x = torch.randn(1, 3, 240, 320, requires_grad=True)
    torch_out = torch.onnx._export(model, x, "saliency_fast.onnx", export_params=True)

In [8]:
onnx_model = onnx.load("saliency_fast.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("saliency_fast.pb")  # export the model

In [9]:
g = tf.GraphDef()
g.ParseFromString(open("saliency_fast.pb", "rb").read())

# g.node
[str(idx)+" "+n.name for (idx, n) in enumerate(g.node)]# if n.name == '0']

['0 Const',
 '1 Const_1',
 '2 Const_2',
 '3 Const_3',
 '4 Const_4',
 '5 Const_5',
 '6 Const_6',
 '7 Const_7',
 '8 Const_8',
 '9 Const_9',
 '10 Const_10',
 '11 Const_11',
 '12 Const_12',
 '13 Const_13',
 '14 Const_14',
 '15 Const_15',
 '16 Const_16',
 '17 Const_17',
 '18 Const_18',
 '19 Const_19',
 '20 Const_20',
 '21 Const_21',
 '22 Const_22',
 '23 Const_23',
 '24 Const_24',
 '25 Const_25',
 '26 Const_26',
 '27 Const_27',
 '28 Const_28',
 '29 Const_29',
 '30 Const_30',
 '31 Const_31',
 '32 Const_32',
 '33 Const_33',
 '34 0',
 '35 transpose/perm',
 '36 transpose',
 '37 Const_34',
 '38 Pad',
 '39 Const_35',
 '40 split/split_dim',
 '41 split',
 '42 transpose_1/perm',
 '43 transpose_1',
 '44 Const_36',
 '45 split_1/split_dim',
 '46 split_1',
 '47 convolution/dilation_rate',
 '48 convolution',
 '49 concat/concat_dim',
 '50 concat',
 '51 Add',
 '52 transpose_2/perm',
 '53 transpose_2',
 '54 Reshape/shape',
 '55 Reshape',
 '56 Reshape_1/shape',
 '57 Reshape_1',
 '58 moments/mean/reduction_ind

In [10]:
# Tensorflow to Tensorflow Lite      IMPORTANT --input_arrays=0 --output_arrays=Sigmoid
!tflite_convert --output_file=saliency_fast.tflite --graph_def_file=saliency_fast.pb --input_arrays=0 --output_arrays=Sigmoid --inference_type=FLOAT


2019-02-13 19:53:44.576925: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-02-13 19:53:44.688703: I tensorflow/core/grappler/devices.cc:53] Number of eligible GPUs (core count >= 8): 0 (Note: TensorFlow was not compiled with CUDA support)
2019-02-13 19:53:44.701355: I tensorflow/core/grappler/clusters/single_machine.cc:359] Starting new session
2019-02-13 19:53:45.004937: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:586] Optimization results for grappler item: graph_to_optimize
2019-02-13 19:53:45.004962: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:588]   model_pruner: Graph size after: 306 nodes (-53), 339 edges (-17), time = 36.476ms.
2019-02-13 19:53:45.004968: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:588]   function_optimizer: Graph size after: 306 nodes (0), 339 edges (0), time = 1.992ms.
2019-02-13 19:53:45.004973: I tensorflow/core/grapple

In [11]:
!tflite_convert --output_file=saliency_fast.tflite --graph_def_file=saliency_fast.pb --input_arrays=0 --output_arrays=Sigmoid --inference_type=QUANTIZED_UINT8  --mean_values=128 --std_dev_values=128 


2019-02-12 11:51:28.589967: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Traceback (most recent call last):
  File "/usr/local/bin/tflite_convert", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/site-packages/tensorflow/lite/python/tflite_convert.py", line 442, in main
    app.run(main=run_main, argv=sys.argv[:1])
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/platform/app.py", line 125, in run
    _sys.exit(main(argv))
  File "/usr/local/lib/python3.6/site-packages/tensorflow/lite/python/tflite_convert.py", line 438, in run_main
    _convert_model(tflite_flags)
  File "/usr/local/lib/python3.6/site-packages/tensorflow/lite/python/tflite_convert.py", line 191, in _convert_model
    output_data = converter.convert()
  File "/usr/local/lib/python3.6/site-packages/tensorflow/lite/python/lite.py", line 500, in convert
    **converter_kwargs)
  F